In [27]:
import pandas as pd
import numpy as np

In [28]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)
train_df.head()

,ID,timestamp,item,corporation,location,supply,price
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [29]:
test_df.head()

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J


In [30]:
in_days = 28
pred_days = 28

seq_data = []

for i in train_df.item.unique():
    now_item = train_df.loc[train_df.item == i]
    for co in train_df.corporation.unique():
        now_co = now_item.loc[now_item.corporation == co]
        for lo in train_df.location.unique():
            now_lo = now_co.loc[now_co.location == lo]

            for i in range(in_days, len(now_lo) - pred_days):
                seq_data.append(list(now_lo.iloc[i]) + list(now_lo.price[i - in_days : i]) + list(now_lo.price[i + 1 : i + 1 + pred_days]))

seq_df = pd.DataFrame(seq_data, columns=[list(train_df.columns) + [f'price_t-{i}' for i in range(in_days, 0, -1)] + [f'price_t + {i}' for i in range(pred_days)]])

In [31]:
first_date_data = []

for i in test_df.item.unique():
    now_item = train_df.loc[train_df.item == i]
    now_item_test = test_df.loc[test_df.item == i]
    for co in test_df.corporation.unique():
        now_co = now_item.loc[now_item.corporation == co]
        now_co_test = now_item_test.loc[now_item_test.corporation == co]
        for lo in test_df.location.unique():
            now_lo = now_co.loc[now_co.location == lo]
            now_lo_test = now_co_test.loc[now_co_test.location == lo]
            
            try:
                first_date_data.append(list(now_lo_test.iloc[0]) + list(now_lo.price[-in_days:]))
            except IndexError:
                continue

test_seq_df = pd.DataFrame(first_date_data, columns=[list(test_df.columns) + [f'price_t-{i}' for i in range(in_days, 0, -1)]])

In [32]:
seq_df.to_csv(f'data/{in_days}-{pred_days}_train.csv', index=False)

In [33]:
test_seq_df.to_csv(f'data/{in_days}_test.csv', index=False)